## 1. Load Data

#### 1.1 Import images

In [1]:
import os
import cv2
import numpy as np

path_img_src = os.path.join(os.getcwd(), 'data/processed')
valid_images = [".jpg",".gif",".png",".tga"]
x = []
for file in os.listdir(path_img_src):
    ext = os.path.splitext(file)[1]
    if ext.lower() not in valid_images: continue
    img = cv2.imread(os.path.join(path_img_src,file), cv2.IMREAD_GRAYSCALE)
    x.append(img)
x = np.array(x)
x = x.reshape(x.shape[0], 1, x.shape[1], x.shape[2])
x.shape

(291, 1, 75, 100)

#### 1.2 Import labels

In [2]:
path_labels_src = os.path.join(os.getcwd(), 'data')
y = []
with open(os.path.join(path_labels_src, 'labels.txt'), "r") as labels_file:
    for line in labels_file:
        y.append(int(line.rstrip('\n')))
y = np.array(y[:x.shape[0]]).reshape(-1,1)
y.shape

(291, 1)

## 2. Preprocess Data

#### 2.1 Normalize images

In [3]:
x = x / 255
np.min(x), np.max(x)

(0.09411764705882353, 1.0)

#### 2.2 Convert labels to one-hot encoding

In [4]:
from keras.utils import to_categorical
print("Before: ", y[0])
y = to_categorical(y)
print("After:  ", y[0])

Using TensorFlow backend.


Before:  [0]
After:   [1. 0. 0. 0. 0. 0. 0. 0. 0.]


#### 2.3 Split data for train and test

In [5]:
seed = 1
np.random.seed(seed)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=np.random)
print("Train Data | Test Data")
print(("{0:^10} | {1:^10}").format(x_train.shape[0], x_test.shape[0]))

Train Data | Test Data
   232     |     59    


## 3. Build Model

#### 3.1 Import Keras and use NCHW mode

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.utils import np_utils

from keras import backend
backend.set_image_dim_ordering('th')

#### 3.2 Create reusable functions

In [7]:
c = x.shape[1]
h = x.shape[2]
w = x.shape[3]
no_of_classes = y.shape[1]
print(x.shape, y.shape)

def train_model(model, epochs=100, batch_size=125):
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)

def evaluate_model(model):
    print("\n\n")
    scores = model.evaluate(x_test, y_test)
    print("Accuracy: ", scores[1]*100, "%")
    return scores

(291, 1, 75, 100) (291, 9)


#### 3.3 Create a baseline model

In [8]:
def baseline_model():
    model = Sequential()
    
    model.add(Conv2D(32, (5, 5), kernel_initializer='normal', activation='relu', input_shape=(c, h, w)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(128, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(no_of_classes, kernel_initializer='normal', activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = baseline_model()
train_model(model)
evaluate_model(model)

Epoch 1/100
232/232 [==============================] - 2s 8ms/step - loss: 2.6140 - acc: 0.1810
Epoch 2/100
232/232 [==============================] - 0s 315us/step - loss: 2.5276 - acc: 0.1422
Epoch 3/100
232/232 [==============================] - 0s 307us/step - loss: 2.2260 - acc: 0.1767
Epoch 4/100
232/232 [==============================] - 0s 298us/step - loss: 2.1186 - acc: 0.1595
Epoch 5/100
232/232 [==============================] - 0s 303us/step - loss: 2.1418 - acc: 0.1853
Epoch 6/100
232/232 [==============================] - 0s 285us/step - loss: 2.1232 - acc: 0.1983
Epoch 7/100
232/232 [==============================] - 0s 307us/step - loss: 2.1027 - acc: 0.1853
Epoch 8/100
232/232 [==============================] - 0s 302us/step - loss: 2.0778 - acc: 0.2241
Epoch 9/100
232/232 [==============================] - 0s 298us/step - loss: 2.0768 - acc: 0.1853
Epoch 10/100
232/232 [==============================] - 0s 302us/step - loss: 2.0577 - acc: 0.2241
Epoch 11/100
232/232 

232/232 [==============================] - 0s 303us/step - loss: 1.7025 - acc: 0.3578
Epoch 84/100
232/232 [==============================] - 0s 294us/step - loss: 1.6863 - acc: 0.3836
Epoch 85/100
232/232 [==============================] - 0s 285us/step - loss: 1.7341 - acc: 0.3448
Epoch 86/100
232/232 [==============================] - 0s 290us/step - loss: 1.6491 - acc: 0.3966
Epoch 87/100
232/232 [==============================] - 0s 311us/step - loss: 1.6959 - acc: 0.3836
Epoch 88/100
232/232 [==============================] - 0s 298us/step - loss: 1.6584 - acc: 0.4095
Epoch 89/100
232/232 [==============================] - 0s 298us/step - loss: 1.6556 - acc: 0.3836
Epoch 90/100
232/232 [==============================] - 0s 307us/step - loss: 1.6781 - acc: 0.3578
Epoch 91/100
232/232 [==============================] - 0s 311us/step - loss: 1.6370 - acc: 0.3793
Epoch 92/100
232/232 [==============================] - 0s 311us/step - loss: 1.6159 - acc: 0.4052
Epoch 93/100
232/232 [=

[1.5156976732157044, 0.4576271236953089]